In [1]:
import numpy as np
from coverage import gen_sats, gen_times, camera_model, get_inst_fov, get_inst_fov2, los_to_earth
from datetime import datetime, timezone, timedelta
import dataclasses
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import branca
import folium

from skyfield.api import load, wgs84, Distance
from skyfield.framelib import itrs
from skyfield.toposlib import ITRSPosition
from landsat import Instrument, Platform, Scene

In [2]:
start_dt = datetime.fromisoformat(Scene.start_utc)
num_days = 2

tles = gen_sats(
    sat_nos=[Platform.norad_id] # How to best handle multiple platforms? (TLE vs. SPG4 model too)
    # sat_nos=[39084]
    # sat_nos=[39084,49260]
)

inst = camera_model(
    name=Instrument.name, 
    fl=Instrument.focal_length_mm, 
    pitch=Instrument.pitch_um*1e-3, 
    h_pix=Instrument.rows, 
    v_pix=Instrument.cols, 
)

times = gen_times(
    start_yr=start_dt.year,
    start_mo=start_dt.month, 
    start_day=start_dt.day, 
    days=num_days, 
    step_min=Instrument.img_period)

xcell_size = ycell_size = .1

Satellite(s) Loaded from TLE:

[<EarthSatellite LANDSAT 9 catalog #49260 epoch 2022-06-15 11:12:37 UTC>]

Propogation time: 
 2022-07-27 00:00:00+00:00 
to 
 2022-07-28 23:59:48+00:00

In [3]:
sat = tles[0][0]

In [4]:
%%timeit
get_inst_fov(sat, times[0], inst)

1.12 ms ± 8.47 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [5]:
llas = get_inst_fov(sat, times[0], inst)

In [6]:
%%timeit
get_inst_fov2(sat, times[0], inst)

1.15 ms ± 7.69 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
llas2 = get_inst_fov2(sat, times[0], inst)

In [8]:
llas2

{'c1': {'lat': -63.38163240859396, 'lon': -12.146297825585277},
 'c2': {'lat': -62.81058302898076, 'lon': -8.709924300718592},
 'c3': {'lat': -61.20749510204343, 'lon': -10.024883795288902},
 'c4': {'lat': -61.7480644487944, 'lon': -13.301729363872125}}

In [9]:
llas

{'c1': {'lat': -63.38163240859396, 'lon': -12.146297825585277},
 'c2': {'lat': -62.81058302898076, 'lon': -8.709924300718592},
 'c3': {'lat': -61.20749510204343, 'lon': -10.024883795288902},
 'c4': {'lat': -61.7480644487944, 'lon': -13.301729363872125}}